In [ ]:
%java -v

In [5]:
from openie import StanfordOpenIE

# https://stanfordnlp.github.io/CoreNLP/openie.html#api
# Default value of openie.affinity_probability_cap was 1/3.
properties = {
    'openie.affinity_probability_cap': 2 / 3,
}

with StanfordOpenIE(properties=properties) as client:
    text = 'Barack Obama was born in Hawaii. Richard Manning wrote this sentence.'
    print('Text: %s.' % text)
    for triple in client.annotate(text):
        print('|-', triple)

    graph_image = 'graph.png'
    client.generate_graphviz_graph(text, graph_image)
    print('Graph generated: %s.' % graph_image)

    with open('/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1/row_0.txt', encoding='utf8') as r:
        corpus = r.read().replace('\n', ' ').replace('\r', '')

    triples_corpus = client.annotate(corpus[0:5000])
    print('Corpus: %s [...].' % corpus[0:80])
    print('Found %s triples in the corpus.' % len(triples_corpus))
    for triple in triples_corpus:
        print('|-', triple)
    print('[...]')

Text: Barack Obama was born in Hawaii. Richard Manning wrote this sentence..
Starting server with command: java -Xmx8G -cp /home/niamatzawad/.stanfordnlp_resources/stanford-corenlp-4.5.3/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-97e51fb33e6d4033.props -preload openie
|- {'subject': 'Barack Obama', 'relation': 'was', 'object': 'born'}
|- {'subject': 'Barack Obama', 'relation': 'was born in', 'object': 'Hawaii'}
|- {'subject': 'Richard Manning', 'relation': 'wrote', 'object': 'sentence'}
Graph generated: graph.png.
Corpus: Radio City is India's first private FM radio station and was started on 3 July 2 [...].
Found 318 triples in the corpus.
|- {'subject': 'India', 'relation': 'has', 'object': 'first private FM radio station'}
|- {'subject': 'Radio City', 'relation': 'is', 'object': "India 's FM radio station"}
|- {'subject': 'Radio City', 'relation': 'was', 'object': 'started'

In [1]:
# needed to load the REBEL model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import math
import torch

# wrapper for wikipedia API
import wikipedia

# scraping of web articles
from newspaper import Article
# google news scraping
from GoogleNews import GoogleNews

# graph visualization
from pyvis.network import Network

# show HTML in notebook
import IPython

from keybert import KeyBERT

from sentence_transformers import SentenceTransformer
import numpy as np
import nltk
import re
from typing import List, Tuple, Union

import spacy

import inspect
import os

import ollama
from tqdm import tqdm
import time
import re
import sys
import pandas as pd

module_path = '../core'
sys.path.append(module_path)

from SemanticSimilarity import SemanticSimilarity
from KeywordExtractor import KeywordExtractor
from TripletGenerator import TripletGenerator

/home/niamatzawad/miniconda3/envs/phd/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)     # Display all rows


In [3]:
class KG:
    def __init__(self,):
        self.triplets = []

    def add_triplets(self, triplets : List):
        self.triplets.extend(triplets)


In [4]:
def retrieve_text_from_filepath(filepath):
    with open(filepath,'r') as f:
        t = f.read()
        return t


In [5]:
%%time
# folder_path = "/home/niamatzawad/niamatzawad/Datasets/UTDBox/demoNY/unsplitted"
    folder_path = "/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1"

s_demo = SemanticSimilarity()
kwe_demo = KeywordExtractor()
kg_demo = KG()

folder_objects = os.listdir(folder_path)
for i, filename in enumerate(folder_objects):
    if os.path.isfile(os.path.join(folder_path, filename)):
        with open(os.path.join(folder_path, filename), 'r') as file:
            text = file.read()
            text_splitted = [text[i * 1900:(i+1) *1900] for i in range(len(text)//1900)]
            start_text_time = time.time()
            for text_s in tqdm(text_splitted):
                start_subtext_time = time.time()
                try:
                    named_entities = kwe_demo.extract_named_entities(text_s)
                except Exception as e:
                    continue
                s_demo.generate_reference(named_entities)
                t = TripletGenerator(text_s, s_demo, kwe_demo, os.path.join(folder_path, filename), named_entities=named_entities)
                triplets = t.get_triplets()
                kg_demo.add_triplets(triplets)
                print(f"Time to create triplets for current subtext - {time.time() - start_subtext_time}")
                
        print(f"Finished processing {i+1}/ {len(folder_objects)} files")
        print(f"Total time taken for current text - {time.time() - start_text_time}")
        print("---")
        


  0%|                                                     | 0/2 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


the text is 
Radio City is India's first private FM radio station and was started on 3 July 2001.  It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).  It plays Hindi, English and regional songs.  It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam October 2007.  Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features.  The Radio station currently plays a mix of Hindi and Regional music.  Abraham Thomas is the CEO of the company./nFootball in Albania existed before the Albanian Football Federation (FSHF) was created.  This was evidenced by the team's registration at the Balkan Cup tournament during 1929-1931, which started in 1929 (although Albania eventually had pressure from the teams becau

 50%|██████████████████████▌                      | 1/2 [00:27<00:27, 27.06s/it]

triplet obtained after the first pass
[('india', ' has', 'radio city'), ('india', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1/row_0.txt'), ('hyderabad', ' where', 'radio city'), ('hyderabad', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1/row_0.txt'), ('mumbai', ' where', 'radio city'), ('mumbai', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1/row_0.txt'), ('hyderabad', ' where', 'radio city'), ('hyderabad', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1/row_0.txt'), ('radio city', ' has', 'company'), ('radio city', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1/row_0.txt'), ('company', ' has', 'company'), ('company', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1/row_0.txt'), ('albania', ' has', ' football federation'), ('albania', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1/row_0.txt'), ('fshf team', ' registered', 'the

100%|█████████████████████████████████████████████| 2/2 [01:01<00:00, 30.95s/it]

none found in line
none found in line
none found in line
triplet obtained after the first pass
[('aria', ' is platinum for', 'the song', 'in the United States and also double platinum by ARIA in Australia.'), ('aria', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1/row_0.txt'), ('aria', ' is platinum for', 'the song', "The song was Warner Bros. Records' fifth-biggest- selling-digital song of 2014, with 1.3 million downloads sold."), ('aria', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1/row_0.txt'), ('warner bros records', ' has as biggest-selling-digital-song', 'the song', 'in the United States and also double platinum by ARIA in Australia.'), ('warner bros records', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1/row_0.txt'), ('aria', ' is platinum for', 'the song', "Women' s colleges in the Southern United States refers to undergraduate, bachelor's degree–granting institutions..."), ('aria', 'filepath', '/home/niama

In [7]:
def get_subjects_from_triplets(triplets):
    return list(set([i[0] for i in triplets]))
get_subjects_from_triplets(kg_demo.triplets)

['women',
 "'defeated'",
 'noah',
 'albania',
 'the teams',
 "'is'",
 'jamie',
 'fshf team',
 "'featured'",
 "'has'",
 'mumbai',
 "'was'",
 'hyderabad',
 'salem college south',
 'chino , california',
 'company',
 "'refer to'",
 'radio city',
 'aria',
 'warner bros records',
 'india',
 "'started'",
 'siblings']

In [8]:
set(kg_demo.triplets)

{("'defeated'",
  " 'regular season'",
  " 'winner'",
  'The 2014–15 Ukrainian Hockey Championship was the 23rd season of the Ukrainian Hockey Championship. Only four teams participated in the league this season, because of the instability in Ukraine and that most of the clubs had economical issues.'),
 ("'defeated'",
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1/row_0.txt'),
 ("'featured'",
  " 'work by'",
  " 'others'",
  'Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others.'),
 ("'featured'",
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1/row_0.txt'),
 ("'has'",
  ' has as biggest-selling-digital-song',
  'the song',
  "in the United States and also double platinum by ARIA in Australia. The song was Warner Bros. Records' fifth-biggest-selling-digital-song of 2014, with 1.3 million downloads sold."),
 ("'has'",
  'filepath',
  '/home/niamatzawad/niamatzawad/Datas

In [8]:
# np.save('disambiguated_reference_list_hotpot_100.npy', s.disambiguated_reference_list)    # .npy extension is added if not given
# disabiguated_reference_liit = np.load('disambiguated_reference_list.npy')

In [9]:
# np.save('embedded_reference_list_hotpot_100.npy', s.embedded_reference)    # .npy extension is added if not given
# embedded_reference = np.load('embedded_reference.npy')

In [9]:
s_demo.disambiguated_reference_list

['radio city',
 'india',
 'it',
 'abraham thomas',
 'company',
 'albania',
 'fshf team',
 'the teams',
 'nechosmith',
 'indie',
 'chino , california',
 'siblings',
 'noah',
 'graham sierota',
 'jamie',
 'echosmith',
 'warner bros records',
 'the riaa',
 'aria',
 'the song',
 "nwomen ' s colleges",
 'the southern united states',
 'women',
 "girls ' seminaries",
 'academies',
 'salem college south',
 'arthur county courthouse',
 'jail',
 'court house',
 'united states s magazine',
 'philadelphia',
 't s . arthur',
 'edgar a . poe',
 'sarah josepha hale',
 "godey ' s lady ' s book",
 'ukrainian',
 'hk kremenchukfirst for women',
 'bauer media']

In [9]:
from py2neo import Graph, Node, Relationship



# Connect to the Neo4j database
graph = Graph("neo4j://localhost:7687", auth=("neo4j", "password"))
graph.delete_all()

In [13]:
from py2neo import Graph, Node, Relationship


# Connect to the Neo4j database
graph = Graph("neo4j://localhost:7687", auth=("neo4j", "password"))
graph.delete_all()

triplets = kg_demo.triplets

# Create the nodes and relationships
for t in triplets:
    # Create the subject node
    subject, predicate, obj = t[0],t[1],t[2]
    context = None
    if len(t) == 4:
        context = t[3]
    
    try:
        n = graph.run(f"MATCH (n:`{subject}`) RETURN n;").data()
    except Exception as e:
        print(f"Problem with subject {subject}")
        continue

    if n:
        subject_node = n[0]["n"]
    else:
        subject_node = Node(subject, id=subject)
        graph.create(subject_node)

    if predicate == "filepath":
        # obj = re.sub("NYT Files", "'NYT Files'",obj)
        obj = "```"+obj+"```"
        context = retrieve_text_from_filepath(obj[3:-3])
        n = graph.run(f"MATCH (n:{obj}) RETURN n;").data()
        if n:
            # print(n[0]["n"])
            object_node = n[0]["n"]
        else:
            # obj = re.sub("'NYT Files'", "NYT Files",obj)[1:-1]
            obj = obj[2:-2]
            object_node = Node(obj, id=obj)
            graph.create(object_node)
            
    else:
        try:
            n = graph.run(f"MATCH (n:`{obj}`) RETURN n;").data() #I had the assumption that subjects are being linked to wrong entities. Seems that is not the case as it exactly matches object if it exists and not the most similar object
        except Exception as e:
            print(f"Problem with object - {obj}")
            continue
            
        if n:
            object_node = n[0]["n"]
        else:
            object_node = Node(obj, id=obj, text=context)
            graph.create(object_node)

    relationship = Relationship(subject_node, predicate, object_node, text=context)
    graph.merge(relationship)

print("Triplets added to the Neo4j database.")

Radio City is India's first private FM radio station and was started on 3 July 2001.  It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).  It plays Hindi, English and regional songs.  It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam October 2007.  Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features.  The Radio station currently plays a mix of Hindi and Regional music.  Abraham Thomas is the CEO of the company./nFootball in Albania existed before the Albanian Football Federation (FSHF) was created.  This was evidenced by the team's registration at the Balkan Cup tournament during 1929-1931, which started in 1929 (although Albania eventually had pressure from the teams because of competi

In [11]:
s = SemanticSimilarity()
s.disambiguated_reference_list

[]

In [20]:
np.save('disambiguated_reference_list_hotpot_4.npy', s_demo.disambiguated_reference_list)    # .npy extension is added if not given
np.save('embedded_reference_kg_demo_hotpot_4.npy', s_demo.embedded_reference)    # .npy extension is added if not given


In [14]:
kg_demo.triplets

[('the ritz - carlton jakarta',
  ' operates',
  'the ritz - carlton jakarta',
  'The Ritz-Carlton Jakarta is operated by The Ritz-Carlton Hotel Company.'),
 ('the ritz - carlton jakarta',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt'),
 ('nthe oberoi family',
  ' is famous for',
  'hotel',
  'The Oberoi family is famous for its involvement in hotels, namely through The Oberoi Group.'),
 ('nthe oberoi family',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt'),
 ('shivaay',
  ' portrays',
  'nthe oberoi family',
  'Nakuul Mehta respectively portrays Shivaay, one of the heirs of the Oberoi family.'),
 ('shivaay',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt'),
 ('the eppley hotel company',
  ' built',
  'hotel',
  'The Hotel Tallcorn was built by the Eppley Hotel Company.'),
 ('the eppley hotel company',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_

In [14]:
triples = []
for k in kg_demo.triplets:
    t = list(k)
    if len(t) == 3:
        t = t + [""]
    if len(t) > 4:
        continue
    triples.append(t)



In [15]:
triples

[['radio city',
  ' was started in',
  'mumbai',
  'It currently plays a mix of Hindi and Regional music.'],
 ['radio city',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1/row_0.txt',
  ''],
 ['radio city',
  ' currently plays',
  ' a mix of Hindi and Regional music',
  'He is the CEO of the company.'],
 ['radio city',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1/row_0.txt',
  ''],
 ['abraham thomas',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1/row_0.txt',
  ''],
 ['albania',
  ' defeated',
  'albania',
  'They started as "Ready Set Go!" until they signed to Warner Bros. Records in May 2012.'],
 ['albania',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1/row_0.txt',
  ''],
 ['echosmith',
  ' was started in',
  'warner bros records',
  'They are best known for their hit song "Cool Kids".'],
 ['echosmith',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_

In [19]:
np.save('hotpot_4_triplets.npy', np.array(triples))